# Простая RNN

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

/home/ilya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# for the sake of reproducibility 

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [4]:
reset_graph()

n_features = 3
n_neurons = 5

# two time steps
X0 = tf.placeholder(dtype=tf.float32, shape=[None, n_features])
X1 = tf.placeholder(dtype=tf.float32, shape=[None, n_features])

Wx = tf.Variable(tf.random_normal(shape=[n_features, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

# tanh as phi
Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

init = tf.global_variables_initializer()

In [5]:
# mini-batches of size 4
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]])  # t = 0
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]])  # t = 1

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [6]:
Y0_val

array([[-0.0664006 ,  0.9625767 ,  0.68105793,  0.7091854 , -0.898216  ],
       [ 0.9977755 , -0.719789  , -0.9965761 ,  0.9673924 , -0.9998972 ],
       [ 0.99999774, -0.99898803, -0.9999989 ,  0.9967762 , -0.9999999 ],
       [ 1.        , -1.        , -1.        , -0.99818915,  0.9995087 ]],
      dtype=float32)

In [7]:
Y1_val

array([[ 1.        , -1.        , -1.        ,  0.4020025 , -0.9999998 ],
       [-0.12210419,  0.62805265,  0.9671843 , -0.9937122 , -0.2583937 ],
       [ 0.9999983 , -0.9999994 , -0.9999975 , -0.85943305, -0.9999881 ],
       [ 0.99928284, -0.99999815, -0.9999058 ,  0.9857963 , -0.92205757]],
      dtype=float32)

# TF

In [8]:
reset_graph()

n_features = 3
n_steps = 2
n_neurons = 5

# rnn model
X = tf.placeholder(tf.float32, [None, n_steps, n_features])
X_seq = tf.unstack(X, axis=1) # shape: [batch_size, i, n_features], total num of i = n_steps

cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
output, state = tf.nn.static_rnn(cell, X_seq, dtype=tf.float32)

output_st = tf.stack(output, axis=1)

init = tf.global_variables_initializer()

In [12]:
X_batch = np.hstack([X0_batch[:, np.newaxis, :], X1_batch[:, np.newaxis, :]]) # X_batch.shape = (4, 2, 3)


with tf.Session() as sess:
    init.run()
    output_st_val = sess.run(output, feed_dict={X: X_batch})

In [13]:
output_st_val

[array([[ 0.30741334, -0.32884315, -0.6542847 , -0.9385059 ,  0.52089024],
        [ 0.99122757, -0.9542541 , -0.7518079 , -0.9995208 ,  0.9820235 ],
        [ 0.9999268 , -0.99783254, -0.8247353 , -0.9999963 ,  0.99947774],
        [ 0.996771  , -0.68750614,  0.8419969 ,  0.9303911 ,  0.8120684 ]],
       dtype=float32),
 array([[ 0.99998885, -0.99976057, -0.0667929 , -0.9999803 ,  0.99982214],
        [-0.6524943 , -0.51520866, -0.37968948, -0.5922594 , -0.08968379],
        [ 0.99862397, -0.99715203, -0.03308626, -0.9991566 ,  0.9932902 ],
        [ 0.99681675, -0.9598194 ,  0.39660627, -0.8307606 ,  0.79671973]],
       dtype=float32)]

# Динамическая RNN

In [ ]:
reset_graph()

n_features = 3
n_steps = 2
n_neurons = 5

# rnn model
X = tf.placeholder(tf.float32, [None, n_steps, n_features])

cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
output, state = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

init = tf.global_variables_initializer()

In [ ]:
X_batch = np.hstack([X0_batch[:, np.newaxis, :], X1_batch[:, np.newaxis, :]])

with tf.Session() as sess:
    init.run()
    output = sess.run(output, feed_dict={X: X_batch})

In [ ]:
output

# LSTM

In [ ]:
import numpy as np
import random
import tensorflow as tf
import tensorflow.contrib.layers as layers

map_fn = tf.map_fn

In [ ]:
def as_bytes(num, final_size):
    res = []
    for _ in range(final_size):
        res.append(num % 2)
        num //= 2
    return res

def generate_example(num_bits):
    a = random.randint(0, 2**(num_bits - 1) - 1)
    b = random.randint(0, 2**(num_bits - 1) - 1)
    res = a + b
    return (as_bytes(a,  num_bits),
            as_bytes(b,  num_bits),
            as_bytes(res,num_bits))

def generate_batch(num_bits, batch_size):
    """Generates instance of a problem.
    Returns
    -------
    x: np.array
        two numbers to be added represented by bits.
        shape: b, i, n
        where:
            b is bit index from the end
            i is example idx in batch
            n is one of [0,1] depending for first and
                second summand respectively
    y: np.array
        the result of the addition
        shape: b, i, n
        where:
            b is bit index from the end
            i is example idx in batch
            n is always 0
    """
    x = np.empty((num_bits, batch_size, 2))
    y = np.empty((num_bits, batch_size, 1))

    for i in range(batch_size):
        a, b, r = generate_example(num_bits)
        x[:, i, 0] = a
        x[:, i, 1] = b
        y[:, i, 0] = r
    return x, y

In [ ]:
INPUT_SIZE = 2  # 2 bits per timestep
RNN_HIDDEN = 20
OUTPUT_SIZE = 1  # 1 bit per timestep
TINY = 1e-6  # to avoid NaNs in logs
LEARNING_RATE = 0.01

inputs  = tf.placeholder(tf.float32, (None, None, INPUT_SIZE))  # (time, batch, in)
outputs = tf.placeholder(tf.float32, (None, None, OUTPUT_SIZE)) # (time, batch, out)

cell = tf.nn.rnn_cell.BasicLSTMCell(RNN_HIDDEN, state_is_tuple=True)

batch_size = tf.shape(inputs)[1]
initial_state = cell.zero_state(batch_size, tf.float32)

rnn_outputs, rnn_states = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, time_major=True)
final_projection = lambda x: layers.linear(x, num_outputs=OUTPUT_SIZE, activation_fn=tf.nn.sigmoid)
predicted_outputs = map_fn(final_projection, rnn_outputs)

error = -(outputs * tf.log(predicted_outputs + TINY) + (1.0 - outputs) * tf.log(1.0 - predicted_outputs + TINY))
error = tf.reduce_mean(error)

train_fn = # AdamOptimizer
accuracy = tf.reduce_mean(tf.cast(tf.abs(outputs - predicted_outputs) < 0.5, tf.float32))

In [ ]:
NUM_BITS = 10
ITERATIONS_PER_EPOCH = 100
BATCH_SIZE = 16

valid_x, valid_y = generate_batch(num_bits=NUM_BITS, batch_size=100)

session = tf.Session()
session.run(tf.initialize_all_variables())

for epoch in range(5):
    epoch_error = 0
    for _ in range(ITERATIONS_PER_EPOCH):
        x, y = generate_batch(num_bits=NUM_BITS, batch_size=BATCH_SIZE)
        epoch_error += session.run([error, train_fn], {
            inputs: x,
            outputs: y,
        })[0]
    epoch_error /= ITERATIONS_PER_EPOCH
    valid_accuracy = session.run(accuracy, {
        inputs:  valid_x,
        outputs: valid_y,
    })
    print ("Epoch %d, train error: %.2f, valid accuracy: %.1f %%" % (epoch, epoch_error, valid_accuracy * 100.0))